<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

## Library installation

Install the libraries we need to fetch market data, compute statistics, and visualize results so this notebook runs cleanly anywhere.

In [ ]:
!pip install yfinance pandas numpy matplotlib

## Imports and setup

We use yfinance to pull historical prices, numpy for fast numerical operations such as annualization, and matplotlib.pyplot to visualize Sharpe behavior over time.

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

Keeping imports minimal helps us focus on the Sharpe workflow without extra dependencies. This mirrors what pros do in quick analyses: load prices, compute stats, and plot diagnostics. It also makes results easier to reproduce across machines and environments.

## Download and prepare daily returns

Pull adjusted close prices for SPY and AAPL over a fixed horizon so we compare assets on the same calendar and sampling frequency.

In [ ]:
data = yf.download(["SPY", "AAPL"], start="2020-01-01", end="2024-12-31")

Using a single source avoids mismatched calendars and survivorship issues that skew risk metrics. Adjusted prices account for splits and dividends, which is important when you want returns that reflect what holders experienced. Consistent sourcing reduces noisy differences that would otherwise leak into Sharpe comparisons.

Convert prices to noncumulative daily returns using close-to-close changes, which is the correct input for Sharpe.

In [ ]:
closes = data.Close
spy_returns = closes.SPY.pct_change().dropna()
aapl_returns = closes.AAPL.pct_change().dropna()

We compute simple daily returns because Sharpe expects a stream of noncumulative observations; the difference versus log returns is negligible at daily horizons. Dropping missing values prevents artificial spikes or divide-by-zero artifacts that can inflate volatility. This gives us a clean, aligned return series for each asset.

## Define and compute Sharpe ratios

Define a helper that computes an annualized Sharpe ratio from daily returns, with an optional constant daily cash adjustment for excess returns.

In [ ]:
def sharpe_ratio(returns, adjustment_factor=0.0):
    """
    Determines the Sharpe ratio of a strategy.

    Parameters
    ----------
    returns : pd.Series or np.ndarray
        Daily returns of the strategy, noncumulative.
    adjustment_factor : int, float
        Constant daily benchmark return throughout the period.

    Returns
    -------
    sharpe_ratio : float

    Note
    -----
    See https://en.wikipedia.org/wiki/Sharpe_ratio for more details.
    """
    returns_risk_adj = returns - adjustment_factor
    return (
        returns_risk_adj.mean() / returns_risk_adj.std()
    ) * np.sqrt(252)

This implementation uses the sample standard deviation (pandas’ default) on noncumulative returns and annualizes with sqrt(252), matching common practice on U.S. trading days. In production, we would subtract a time-varying daily cash rate rather than a constant when rates move. The function enforces the “excess return per unit of volatility” idea that lets us compare assets fairly.

Compute full-sample Sharpe ratios for SPY and AAPL to benchmark their risk-adjusted performance over the same window.

In [ ]:
sharpe_ratio(spy_returns)

In [ ]:
sharpe_ratio(aapl_returns)

Looking at full-sample Sharpe gives a quick first pass on which asset delivered more return per unit of risk, unlevered and on identical sampling. This avoids the “bigger CAGR wins” trap by normalizing for volatility and frequency. Treat these as starting points before you inspect stability through time.

## Visualize rolling Sharpe and differences

Track 30-day rolling Sharpe for AAPL, then plot its distribution and the rolling Sharpe difference versus SPY to see stability and relative edge.

In [ ]:
aapl_returns.rolling(30).apply(sharpe_ratio).plot()

In [ ]:
aapl_returns.rolling(30).apply(sharpe_ratio).hist(bins=50)

In [ ]:
(
    aapl_returns.rolling(30).apply(sharpe_ratio)
    - spy_returns.rolling(30).apply(sharpe_ratio)
).hist(bins=50)

Rolling windows reveal regime shifts that a single full-period number hides, which is how desks set guardrails and monitor drift. The histograms show how often AAPL’s efficiency clusters at certain levels and when it outperforms SPY on a risk-adjusted basis. Short windows can be noisy, so in real reviews we also test multiple horizons and subtract a realistic daily cash rate when conditions change.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.